In [1]:
#Intalacion de la librería opencv si es que no se tienen
!pip install opencv-python

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [2]:
!pip freeze >requirements.txt

In [ ]:
#Librerias utilizadas en el proyecto:
import cv2
import matplotlib.pyplot as plt
from IPython.display import clear_output
import pandas as pd
import numpy as np
import math